<a href="https://colab.research.google.com/github/francescopatane96/Computer_aided_drug_discovery_kit/blob/main/ML_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install rdkit   #install rdkit library

In [ ]:
!pip install lazypredict  #install lazypredict

In [ ]:
!pip install git+https://github.com/volkamerlab/teachopencadd.git  #install teachopencadd dependecies 

In [ ]:
from pathlib import Path
import seaborn as sns
from sklearn.metrics import confusion_matrix
from warnings import filterwarnings
import time
import lazypredict
from lazypredict.Supervised import LazyRegressor
from lazypredict.Supervised import LazyClassifier

import pandas as pd
from sklearn.ensemble import RandomForestRegressor
import numpy as np
from sklearn import svm, metrics, clone
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import auc, accuracy_score, recall_score
from sklearn.metrics import roc_curve, roc_auc_score
import matplotlib.pyplot as plt
from rdkit import Chem
from rdkit.Chem import MACCSkeys
from rdkit.Chem.AllChem import GetMorganFingerprintAsBitVect

from sklearn.feature_selection import SelectFromModel
from teachopencadd.utils import seed_everything

# Silence some expected warnings
filterwarnings("ignore")
# Fix seed for reproducible results
SEED = 44
seed_everything(SEED)

In [ ]:
# Read data (Lipinski)
chembl_df1 = pd.read_csv(
    "IDH_compounds_lipinski.csv",    #read lipinski's descriptors csv
    index_col=0
)

# Look at head
print("Shape of dataframe : ", chembl_df1.shape)
chembl_df1.head()


In [ ]:
# remove NaN, if present
chembl_df2 = chembl_df1.dropna()

In [ ]:
chembl_df2.shape

In [ ]:
# Keep only the columns we want
chembl_df3 = chembl_df2[["molecule_chembl_id", "smiles", "pIC50"]]
chembl_df3.head()


In [ ]:
chembl_df4 = chembl_df3.reset_index()
chembl_df4

In [ ]:
def filter(IC):
  if IC <= 5.5:
    return 'low'
  if (IC > 5.5 and IC <= 7.5):
    return 'medium'
  if IC > 7.5:
    return 'high'

chembl_df4['bio_class'] = chembl_df4['pIC50'].apply(filter)
chembl_df4.head(100)

install padel (descriptors calculator)

In [ ]:
! wget https://github.com/gromdimon/features/raw/main/padel.sh
! wget https://github.com/gromdimon/features/raw/main/padel.zip

In [ ]:
!unzip padel.zip

In [ ]:
selection = ['smiles', 'molecule_chembl_id']     #select columns we want to retain
act_selected = chembl_df4[selection]
act_selected.to_csv('molecule.smi', sep='\t', index=False, header=False )

In [ ]:
! cat molecule.smi | head -5
! cat molecule.smi | wc -l

In [ ]:
!cat padel.sh   #read the script

In [ ]:
!bash padel.sh    #run padel (it reads molecule.smi file that contains canonical form smiles)

In [ ]:
actx = pd.read_csv('descriptors_output.csv') #padel generates a file called 'descriptors_output.csv'
actx                                         

In [ ]:
actx_final = actx.drop('Name', axis=1)
actx_final

In [ ]:
X = actx_final                  #descriptors
Y = chembl_df4.bio_class  #bioactivity class

random forest and feature selection

In [ ]:
# Spliting data in 80\20 ratio
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.3, random_state=44)

In [ ]:
# Seeing the data that was prepared
X_train.shape, Y_train.shape, X_test.shape, Y_test.shape

In [ ]:
reg = SelectFromModel(RandomForestClassifier(n_estimators = 100))
reg.fit(X_train, Y_train)

In [ ]:
reg.get_support()

In [ ]:
selected_feat= X_train.columns[(reg.get_support())]
len(selected_feat)

In [ ]:
print(selected_feat)

In [ ]:
# Defines and builds the lazyclassifier
reg = LazyRegressor(verbose=0,ignore_warnings=True, custom_metric=None)
models_train,predictions_train = reg.fit(X_train, X_train, Y_train, Y_train)

In [ ]:
# Performance table of the training set (80% subset)
models_train

In [ ]:
# Checking the study on a test sample
reg = LazyRegressor(verbose=0,ignore_warnings=True, custom_metric=None)
models_test,predictions_test = reg.fit(X_train,X_test,Y_train,Y_test)

In [ ]:
models_test

In [ ]:
X_train = X_train.astype('int32')
Y_train = Y_train.astype('string')

In [ ]:
model = RandomForestClassifier(n_estimators=100, random_state=44)
model.fit(X_train, Y_train)
r2 = model.score(X_test, Y_test)
r2                                 #show explained variance

In [ ]:
# View accuracy score
accuracy_score(Y_test, Y_pred)

In [ ]:
# Try data with test sample

Y_pred = model.predict(X_test)
print(Y_pred)

Accuracy score is not a great measure of classifier performance when the classes are imbalanced. 

We need more information to define how well the model really performed. 

Did it perform equally well for each class? Were there any pairs of classes it found especially hard to distinguish? Let's find out with a confusion matrix.

the rows represent the true labels and the columns represent predicted labels. Values on the diagonal represent the number (or percent, in a normalized confusion matrix) of times where the predicted label matches the true label. Values in the other cells represent instances where the classifier mislabeled an observation; the column tells us what the classifier predicted, and the row tells us what the right label was. This is a convenient way to spot areas where the model may need a little extra training.

In [ ]:
confusion_matrix(Y_test, Y_pred)

In [ ]:
# Get and reshape confusion matrix data
matrix = confusion_matrix(Y_test, Y_pred)
matrix = matrix.astype('float') / matrix.sum(axis=1)[:, np.newaxis]

# Build the plot
plt.figure(figsize=(16,7))
sns.set(font_scale=1.4)
sns.heatmap(matrix, annot=True, annot_kws={'size':10},
            cmap=plt.cm.Greens, linewidths=0.2)

# Add labels to the plot
class_names = ['low', 'medium', 'high']
tick_marks = np.arange(len(class_names))
tick_marks2 = tick_marks + 0.5
plt.xticks(tick_marks, class_names, rotation=25)
plt.yticks(tick_marks2, class_names, rotation=0)
plt.xlabel('Predicted label')
plt.ylabel('True label')
plt.title('Confusion Matrix for Random Forest Model')
plt.show()

Our classifier struggled at predicting the medium and low labels. about half times, medium and low were mislabeled as high bioactivity.

This example demonstrates the use of permutation_test_score to evaluate the significance of a cross-validated score using permutations.

- We will also generate some random feature data (i.e., 20 features), uncorrelated with the class labels in the dataset.

- Next, we calculate the permutation_test_score using the original dataset, which strongly predict the labels and the randomly generated features and iris labels, which should have no dependency between features and labels. We use the SVC classifier and Accuracy score to evaluate the model at each round.

- permutation_test_score generates a null distribution by calculating the accuracy of the classifier on 1000 different permutations of the dataset, where features remain the same but labels undergo different permutations. This is the distribution for the null hypothesis which states there is no dependency between the features and labels. An empirical p-value is then calculated as the percentage of permutations for which the score obtained is greater that the score obtained using the original data.

In [ ]:
n_uncorrelated_features = 20
rng = np.random.RandomState(seed=44)
# Use same number of samples as in iris and 20 features
X_rand = rng.normal(size=(X.shape[0], n_uncorrelated_features))

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import permutation_test_score

cv = StratifiedKFold(5, shuffle=True, random_state=44)

score, perm_scores, pvalue = permutation_test_score(
    model, X, Y, scoring="accuracy", cv=cv, n_permutations=100
)

score_rand, perm_scores_rand, pvalue_rand = permutation_test_score(
    model, X_rand, Y, scoring="accuracy", cv=cv, n_permutations=100
)

In [ ]:
fig, ax = plt.subplots()

ax.hist(perm_scores, bins=20, density=True)
ax.axvline(score, ls="--", color="r")
score_label = f"Score on original\ndata: {score:.2f}\n(p-value: {pvalue:.3f})"
ax.text(0.7, 10, score_label, fontsize=12)
ax.set_xlabel("Accuracy score")
_ = ax.set_ylabel("Probability")

Below we plot a histogram of the permutation scores (the null distribution). The red line indicates the score obtained by the classifier on the original data. The score is much better than those obtained by using permuted data and the p-value is thus very low. This indicates that there is a low likelihood that this good score would be obtained by chance alone. It provides evidence that the dataset contains real dependency between features and labels and the classifier was able to utilize this to obtain good results.

Below we plot the null distribution for the randomized data. The permutation scores are similar to those obtained using the original iris dataset because the permutation always destroys any feature label dependency present. The score obtained on the original randomized data in this case though, is very poor. This results in a large p-value, confirming that there was no feature label dependency in the original data.

In [ ]:
fig, ax = plt.subplots()

ax.hist(perm_scores_rand, bins=20, density=True)
ax.set_xlim(0.13)
ax.axvline(score_rand, ls="--", color="r")
score_label = f"Score on original\ndata: {score_rand:.2f}\n(p-value: {pvalue_rand:.3f})"
ax.text(0.14, 7.5, score_label, fontsize=12)
ax.set_xlabel("Accuracy score")
ax.set_ylabel("Probability")
plt.show()

generate model as joblib Object

In [ ]:
import joblib

In [ ]:
joblib.dump(model, "./random_forest.joblib")    #save the model

In [ ]:
loaded_rf = joblib.load("./random_forest.joblib")    #load the model

In [ ]:
loaded_rf.predict(actx_df)              #predict pIC50

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.set(color_codes=True)
sns.set_style("white")

ax = sns.regplot(Y_test, Y_pred, scatter_kws={'alpha':0.4, 'color': 'black'}, line_kws={'color': 'red'})
ax.set_xlabel('Experimental pIC50', fontsize='large', fontweight='bold', color='red')
ax.set_ylabel('Predicted pIC50', fontsize='large', fontweight='bold', color='red')
ax.set_xlim(0, 12)
ax.set_ylim(0, 12)
ax.figure.set_size_inches(10, 10)
plt.show

In [ ]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model, X, Y, cv=12)
print(scores)

[0.5503876  0.78294574 0.75193798 0.75968992 0.82945736 0.78294574
 0.7751938  0.69767442 0.68217054 0.734375   0.671875   0.6015625 ]


In [ ]:
r2_training = model.score(X_train, Y_train)
print(r2_training)
r2_test = model.score(X_test, Y_test)
print(r2_test)

0.9472710453283997
0.7198275862068966
